#### Trabajo sobre la base de datos de la **"Encyclopaedia of exoplanetary systems"** de la Unión Europea

##### En lo que sigue se propone trabajar con bases de datos en observatorios virtuales: usando un catálogo de exoplanetas de la base de datos [exoplanets.eu](https://www.exoplanets.eu)

**Enlaces útiles:**
- [Encyclopaedia of exoplanetary systems](https://www.exoplanets.eu) - Base de datos principal
- [Catálogo CSV](https://www.exoplanets.eu/catalog/csv/) - Descarga de datos en formato CSV
- [Documentación de la API](https://www.exoplanets.eu/api/) - Para acceso programático

In [18]:
# Verificación de instalación de pyvo

import pyvo
print(f"PyVO version: {pyvo.__version__}")

PyVO version: 1.5.2


In [19]:
# Seleccionamos el servicio de exoplanetas
# Usamos la URL correcta del servicio TAP de Paris Observatory

service = pyvo.dal.TAPService("http://voparis-tap-planeto.obspm.fr/tap")

# Verificamos que el servicio esté funcionando
print("Conectando al servicio TAP...")
print(f"URL del servicio: {service.baseurl}")

# Listamos las tablas disponibles usando una consulta ADQL
try:
    print("Obteniendo lista de tablas...")
    
    # Consulta para obtener información de las tablas
    table_query = "SELECT table_name, table_type FROM TAP_SCHEMA.tables"
    table_results = service.search(table_query)
    
    print(f"Número de tablas disponibles: {len(table_results)}")
    print("\nPrimeras tablas encontradas:")
    
    # Convertir a lista para mejor manejo
    tables_list = []
    for row in table_results:
        tables_list.append({
            'name': str(row['table_name']), 
            'type': str(row['table_type'])
        })
    
    # Mostrar primeras 10 tablas
    for i, table in enumerate(tables_list[:10]):
        print(f"- {table['name']} ({table['type']})")
    
    print("\nBuscando tablas relacionadas con exoplanetas...")
    exo_tables = [table for table in tables_list if 'exoplanet' in table['name'].lower() or 'planet' in table['name'].lower()]
    
    if exo_tables:
        print("Tablas de exoplanetas encontradas:")
        for table in exo_tables:
            print(f"- {table['name']}")
    else:
        print("No se encontraron tablas específicas de exoplanetas en los nombres.")
        print("\nMostrando todas las tablas disponibles:")
        for table in tables_list:
            print(f"- {table['name']}")
        
except Exception as e:
    print(f"Error al obtener tablas: {e}")
    print("Intentaremos con una consulta más básica...")

Conectando al servicio TAP...
URL del servicio: http://voparis-tap-planeto.obspm.fr/tap
Obteniendo lista de tablas...
Número de tablas disponibles: 41

Primeras tablas encontradas:
- tap_schema.schemas (table)
- tap_schema.tables (table)
- tap_schema.columns (table)
- tap_schema.keys (table)
- tap_schema.key_columns (table)
- tap_schema.groups (table)
- iks.epn_core (table)
- basecom.epn_core (table)
- exoplanet.epn_core (table)
- vims_satellites.epn_core (table)

Buscando tablas relacionadas con exoplanetas...
Tablas de exoplanetas encontradas:
- exoplanet.epn_core
- planets.epn_core
- spectro_planets.epn_core
- hst_planeto.epn_core
Número de tablas disponibles: 41

Primeras tablas encontradas:
- tap_schema.schemas (table)
- tap_schema.tables (table)
- tap_schema.columns (table)
- tap_schema.keys (table)
- tap_schema.key_columns (table)
- tap_schema.groups (table)
- iks.epn_core (table)
- basecom.epn_core (table)
- exoplanet.epn_core (table)
- vims_satellites.epn_core (table)

Buscand

In [20]:
# Realizamos un query para obtener información sobre exoplanetas
# Usamos la tabla exoplanet.epn_core que encontramos

print("Preparando consulta para la tabla de exoplanetas...")

# Primero, veamos qué columnas tiene la tabla exoplanet.epn_core
try:
    columns_query = "SELECT column_name, datatype FROM TAP_SCHEMA.columns WHERE table_name = 'exoplanet.epn_core'"
    columns_result = service.search(columns_query)
    
    print("Columnas disponibles en exoplanet.epn_core:")
    columns_list = []
    for row in columns_result:
        column_info = f"{row['column_name']} ({row['datatype']})"
        columns_list.append(str(row['column_name']))
        print(f"- {column_info}")
    
    print(f"\nTotal de columnas: {len(columns_list)}")
    
    # Crear una consulta usando algunas columnas principales
    if 'target_name' in columns_list:
        query = "SELECT TOP 10 target_name, target_class FROM exoplanet.epn_core"
    else:
        # Consulta más genérica
        query = "SELECT TOP 10 * FROM exoplanet.epn_core"
    
    print(f"Query a ejecutar: {query}")
    
except Exception as e:
    print(f"Error al obtener columnas: {e}")
    # Consulta de fallback
    query = "SELECT TOP 5 * FROM exoplanet.epn_core"
    print(f"Usando consulta de fallback: {query}")

Preparando consulta para la tabla de exoplanetas...
Columnas disponibles en exoplanet.epn_core:
- granule_uid (char)
- granule_gid (char)
- obs_id (char)
- dataproduct_type (char)
- target_name (char)
- target_class (char)
- time_min (double)
- time_max (double)
- time_sampling_step_min (double)
- time_sampling_step_max (double)
- time_exp_min (double)
- time_exp_max (double)
- spectral_range_min (double)
- spectral_range_max (double)
- spectral_sampling_step_min (double)
- spectral_sampling_step_max (double)
- spectral_resolution_min (double)
- spectral_resolution_max (double)
- c1min (double)
- c1max (double)
- c2min (double)
- c2max (double)
- c3min (double)
- c3max (double)
- s_region (char)
- c1_resol_min (double)
- c1_resol_max (double)
- c2_resol_min (double)
- c2_resol_max (double)
- c3_resol_min (double)
- c3_resol_max (double)
- spatial_frame_type (char)
- incidence_min (double)
- incidence_max (double)
- emergence_min (double)
- emergence_max (double)
- phase_min (double)
- 

In [21]:
# Ejecutamos el query y mostramos los resultados
try:
    print("Ejecutando consulta...")
    results = service.search(query)
    
    print(f"Consulta exitosa. Número de filas encontradas: {len(results)}")
    
    if len(results) > 0:
        print("\nPrimeros resultados:")
        print("-" * 50)
        
        # Mostrar información de las columnas
        if hasattr(results, 'fieldnames'):
            print(f"Columnas disponibles: {results.fieldnames}")
            print()
        
        # Mostrar las primeras filas (manejo correcto)
        count = 0
        for row in results:
            if count >= 5:  # Limitar a 5 filas
                break
            print(f"Fila {count+1}:")
            # Acceder a los datos por nombre de columna
            if 'target_name' in results.fieldnames:
                print(f"  Nombre: {row['target_name']}")
                print(f"  Clase: {row['target_class']}")
            else:
                # Si no podemos acceder por nombre, mostrar toda la fila
                print(f"  {row}")
            print()
            count += 1
    else:
        print("No se encontraron resultados para la consulta.")
        
except Exception as e:
    print(f"Error al ejecutar la consulta: {e}")
    print("Intentemos una consulta más básica...")
    
    # Consulta alternativa para verificar la conectividad
    try:
        basic_query = "SELECT TOP 3 table_name FROM TAP_SCHEMA.tables"
        basic_results = service.search(basic_query)
        print(f"\nConsulta básica exitosa. Tablas encontradas:")
        for row in basic_results:
            print(f"- {row['table_name']}")
    except Exception as e2:
        print(f"Error en consulta básica: {e2}")

Ejecutando consulta...
Consulta exitosa. Número de filas encontradas: 10

Primeros resultados:
--------------------------------------------------
Columnas disponibles: ('target_name', 'target_class')

Fila 1:
  Nombre: 109 Psc b
  Clase: exoplanet

Fila 2:
  Nombre: 112 Psc b
  Clase: exoplanet

Fila 3:
  Nombre: 112 Psc c
  Clase: exoplanet

Fila 4:
  Nombre: 11 Com Ab
  Clase: exoplanet

Fila 5:
  Nombre: 11 UMi b
  Clase: exoplanet

Consulta exitosa. Número de filas encontradas: 10

Primeros resultados:
--------------------------------------------------
Columnas disponibles: ('target_name', 'target_class')

Fila 1:
  Nombre: 109 Psc b
  Clase: exoplanet

Fila 2:
  Nombre: 112 Psc b
  Clase: exoplanet

Fila 3:
  Nombre: 112 Psc c
  Clase: exoplanet

Fila 4:
  Nombre: 11 Com Ab
  Clase: exoplanet

Fila 5:
  Nombre: 11 UMi b
  Clase: exoplanet



In [22]:
# ALTERNATIVA: Usar NASA Exoplanet Archive si el servicio anterior no funciona
import requests
import pandas as pd
from io import StringIO

def get_exoplanets_nasa():
    """
    Función para obtener datos de exoplanetas desde NASA Exoplanet Archive
    """
    try:
        # URL del NASA Exoplanet Archive
        nasa_url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"
        
        # Query ADQL para obtener algunos exoplanetas
        query = """
        SELECT TOP 10 
            pl_name, 
            hostname, 
            pl_orbper, 
            pl_bmasse, 
            pl_rade, 
            st_dist,
            disc_year
        FROM ps 
        WHERE pl_orbper IS NOT NULL 
        AND pl_bmasse IS NOT NULL
        ORDER BY disc_year DESC
        """
        
        # Parámetros para la petición
        params = {
            'REQUEST': 'doQuery',
            'LANG': 'ADQL',
            'FORMAT': 'csv',
            'QUERY': query
        }
        
        print("Conectando al NASA Exoplanet Archive...")
        response = requests.get(nasa_url, params=params, timeout=30)
        
        if response.status_code == 200:
            # Convertir a DataFrame de pandas
            df = pd.read_csv(StringIO(response.text))
            print(f"✓ Datos obtenidos exitosamente: {len(df)} exoplanetas")
            return df
        else:
            print(f"Error HTTP: {response.status_code}")
            return None
            
    except Exception as e:
        print(f"Error al conectar con NASA Archive: {e}")
        return None

# Ejecutar la función alternativa
print("=== OPCIÓN ALTERNATIVA: NASA EXOPLANET ARCHIVE ===")
nasa_data = get_exoplanets_nasa()

if nasa_data is not None:
    print("\nInformación de los datos:")
    print(f"Columnas: {list(nasa_data.columns)}")
    print(f"Número de filas: {len(nasa_data)}")
    print("\nPrimeros 5 exoplanetas:")
    print(nasa_data.head())

=== OPCIÓN ALTERNATIVA: NASA EXOPLANET ARCHIVE ===
Conectando al NASA Exoplanet Archive...
Error HTTP: 400


In [23]:
# Consultas más avanzadas sobre exoplanetas
print("=== ANÁLISIS AVANZADO DE EXOPLANETAS ===\n")

# 1. Obtener exoplanetas con información orbital completa
print("1. Exoplanetas con datos orbitales completos:")
orbital_query = """
SELECT TOP 15 
    target_name, 
    star_name,
    mass, 
    radius, 
    period, 
    semi_major_axis,
    star_distance,
    discovered
FROM exoplanet.epn_core 
WHERE mass IS NOT NULL 
    AND radius IS NOT NULL 
    AND period IS NOT NULL 
    AND semi_major_axis IS NOT NULL
ORDER BY discovered DESC
"""

try:
    orbital_results = service.search(orbital_query)
    print(f"Encontrados {len(orbital_results)} exoplanetas con datos completos\n")
    
    for i, planet in enumerate(orbital_results):
        if i >= 5:  # Mostrar solo los primeros 5
            break
        print(f"{i+1}. {planet['target_name']}")
        print(f"   Estrella: {planet['star_name']}")
        print(f"   Masa: {planet['mass']:.3f} M⊕")
        print(f"   Radio: {planet['radius']:.3f} R⊕")
        print(f"   Período: {planet['period']:.2f} días")
        print(f"   Semieje mayor: {planet['semi_major_axis']:.3f} AU")
        print(f"   Distancia estelar: {planet['star_distance']:.1f} pc")
        print(f"   Descubierto: {planet['discovered']}")
        print()
        
except Exception as e:
    print(f"Error en consulta orbital: {e}")

# 2. Estadísticas básicas
print("\n2. Estadísticas de la base de datos:")
stats_query = "SELECT COUNT(*) as total_exoplanets FROM exoplanet.epn_core"

try:
    stats_results = service.search(stats_query)
    total = stats_results[0]['total_exoplanets']
    print(f"Total de exoplanetas en la base de datos: {total}")
    
    # Contar por clase
    class_query = "SELECT target_class, COUNT(*) as count FROM exoplanet.epn_core GROUP BY target_class"
    class_results = service.search(class_query)
    
    print("\nDistribución por clase:")
    for row in class_results:
        print(f"- {row['target_class']}: {row['count']} objetos")
        
except Exception as e:
    print(f"Error en estadísticas: {e}")

print("\n✓ Análisis completado!")

=== ANÁLISIS AVANZADO DE EXOPLANETAS ===

1. Exoplanetas con datos orbitales completos:
Encontrados 15 exoplanetas con datos completos

1. TOI-1743 b
   Estrella: TOI-1743
   Masa: 0.015 M⊕
   Radio: 0.163 R⊕
   Período: 4.27 días
   Semieje mayor: 0.036 AU
   Distancia estelar: 41.3 pc
   Descubierto: 2025

2. TOI-2031 Ab
   Estrella: TOI-2031 A
   Masa: 0.800 M⊕
   Radio: 1.267 R⊕
   Período: 5.72 días
   Semieje mayor: 0.066 AU
   Distancia estelar: 276.4 pc
   Descubierto: 2025

3. TOI-1203 Ab
   Estrella: TOI-1203 A
   Masa: 0.011 M⊕
   Radio: 0.136 R⊕
   Período: 4.16 días
   Semieje mayor: 0.049 AU
   Distancia estelar: 65.0 pc
   Descubierto: 2025

4. TOI-1203 Ad
   Estrella: TOI-1203 A
   Masa: 0.023 M⊕
   Radio: 0.260 R⊕
   Período: 25.50 días
   Semieje mayor: 0.163 AU
   Distancia estelar: 65.0 pc
   Descubierto: 2025

5. TOI-1846 b
   Estrella: TOI-1846
   Masa: 0.014 M⊕
   Radio: 0.160 R⊕
   Período: 3.93 días
   Semieje mayor: 0.036 AU
   Distancia estelar: 47.2 pc
   De